In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
SDPA_DROPOUT = nn.Dropout(0.1)

In [3]:
def sdpa_forward(self, temperature, q, k, v, mask=None):

        attn = torch.matmul(q / temperature, k.transpose(2, 3))

        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)

        attn = SDPA_DROPOUT(F.softmax(attn, dim=-1))
        output = torch.matmul(attn, v)

        return output, attn

In [ ]:
{"hidden_size": 768, "hidden_act": "gelu", "initializer_range": 0.02, "vocab_size": 30522, "hidden_dropout_prob": 0.1, "num_attention_heads": 12, "type_vocab_size": 2, "max_position_embeddings": 512, "num_hidden_layers": 12, "intermediate_size": 3072, "attention_probs_dropout_prob": 0.1}

In [ ]:
N_HEAD = 12
M_SIZE = 768
K_SIZE = 768/12
V_SIZE = 768/12
w_qs = nn.Linear(M_SIZE, N_HEAD * K_SIZE, bias=False)
w_ks = nn.Linear(M_SIZE, N_HEAD * K_SIZE, bias=False)
w_vs = nn.Linear(M_SIZE, N_HEAD * V_SIZE, bias=False)
fc = nn.Linear(N_HEAD * V_SIZE, M_SIZE, bias=False)
#attention
DROPOUT = nn.Dropout(0.1)
LAYER_NORM = nn.LayerNorm(M_SIZE, eps=1e-6)

In [ ]:
MultiHeadAttention(12, 768, 768/12, 768/12, 0.1)

In [ ]:
self.n_head = n_head
self.k_size = k_size
self.v_size = v_size

self.w_qs = nn.Linear(d_model, n_head * k_size, bias=False)
self.w_ks = nn.Linear(d_model, n_head * k_size, bias=False)
self.w_vs = nn.Linear(d_model, n_head * v_size, bias=False)
self.fc = nn.Linear(n_head * v_size, d_model, bias=False)

self.attention = ScaledDotProductAttention(temperature=k_size ** 0.5)

self.dropout = nn.Dropout(dropout)
self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)

In [ ]:
k_size, v_size, n_head = self.k_size, self.v_size, self.n_head
sz_b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)

residual = q

# Pass through the pre-attention projection: b x lq x (n*dv)
# Separate different heads: b x lq x n x dv
q = self.w_qs(q).view(sz_b, len_q, n_head, k_size)
k = self.w_ks(k).view(sz_b, len_k, n_head, k_size)
v = self.w_vs(v).view(sz_b, len_v, n_head, v_size)

# Transpose for attention dot product: b x n x lq x dv
q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

if mask is not None:
	mask = mask.unsqueeze(1)   # For head axis broadcasting.

q, attn = self.attention(q, k, v, mask=mask)

# Transpose to move the head dimension back: b x lq x n x dv
# Combine the last two dimensions to concatenate all the heads together: b x lq x (n*dv)
q = q.transpose(1, 2).contiguous().view(sz_b, len_q, -1)
q = self.dropout(self.fc(q))
q += residual

q = self.layer_norm(q)


In [ ]:
class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, k_size, v_size, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.k_size = k_size
        self.v_size = v_size

        self.w_qs = nn.Linear(d_model, n_head * k_size, bias=False)
        self.w_ks = nn.Linear(d_model, n_head * k_size, bias=False)
        self.w_vs = nn.Linear(d_model, n_head * v_size, bias=False)
        self.fc = nn.Linear(n_head * v_size, d_model, bias=False)

        self.attention = ScaledDotProductAttention(temperature=k_size ** 0.5)

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, q, k, v, mask=None):

        k_size, v_size, n_head = self.k_size, self.v_size, self.n_head
        sz_b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)

        residual = q

        # Pass through the pre-attention projection: b x lq x (n*dv)
        # Separate different heads: b x lq x n x dv
        q = self.w_qs(q).view(sz_b, len_q, n_head, k_size)
        k = self.w_ks(k).view(sz_b, len_k, n_head, k_size)
        v = self.w_vs(v).view(sz_b, len_v, n_head, v_size)

        # Transpose for attention dot product: b x n x lq x dv
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        if mask is not None:
            mask = mask.unsqueeze(1)   # For head axis broadcasting.

        q, attn = self.attention(q, k, v, mask=mask)

        # Transpose to move the head dimension back: b x lq x n x dv
        # Combine the last two dimensions to concatenate all the heads together: b x lq x (n*dv)
        q = q.transpose(1, 2).contiguous().view(sz_b, len_q, -1)
        q = self.dropout(self.fc(q))
        q += residual

        q = self.layer_norm(q)

        return q, attn